# <a id='toc1_'></a>[Model Evaluation](#toc0_)

This notebook contains the code to evaluate the models on the test set.

**Table of contents**<a id='toc0_'></a>    
- [Model Evaluation](#toc1_)    
- [Import libraries](#toc2_)    
- [Import data](#toc3_)    
- [Prepare data for training](#toc4_)    
  - [Neural network data](#toc4_1_)    
  - [Linear model data](#toc4_2_)    
- [Train Neural Network](#toc5_)    
  - [Graph the model](#toc5_1_)    
- [Train linear models](#toc6_)    
  - [OLS](#toc6_1_)    
  - [LASSO](#toc6_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Import libraries](#toc0_)

In [31]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.compose import ColumnTransformer
# import shap
import matplotlib.pyplot as plt
# from matplotlib.colors import TwoSlopeNorm
# import math
# import itertools
import warnings
from sklearn.exceptions import ConvergenceWarning

from libs.models import *
from libs.functions import *

plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 4), 'figure.dpi': 300})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# <a id='toc3_'></a>[Import data](#toc0_)

In [32]:
# read data
df = pd.read_csv('data/data.csv')

# <a id='toc4_'></a>[Prepare data for training](#toc0_)

In [ ]:
# identify dummy vs. numeric columns
feature_cols = [col for col in df.columns if col not in ['timestamp', 'ticker', 'target']]
nace_cols = [c for c in feature_cols if c.startswith('NACE_')]
dummy_cols = ['divi','divo'] # sin removed
macro_cols = ['discount', 'tms', 'dp', 'ep', 'svar']

# nummeric cols = cols not in cat and macro cols
numeric_cols = [c for c in feature_cols if c not in dummy_cols and c not in nace_cols and c not in macro_cols]

# feature_cols = numeric_cols + dummy_cols + nace_cols # reorder columns to have numeric first

df_raw = df.copy(deep=True)
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])

# drop data from 2025
df_raw = df_raw[df_raw['timestamp'] < '2025-01-01']

# prepare expanding window splits
periods = {
    '21' : '2020-01-01', # 2021 is the test set
    '22' : '2021-01-01', # 2022 is the test set
    '23' : '2022-01-01', # 2023 is the test set
    '24': '2023-01-01' # 2024 is the test set
}

y_values = df_raw['target'].values.astype('float32')

In [34]:
C = df[numeric_cols + dummy_cols].values         # shape = (n_rows, P_c)
X = df[macro_cols].values           # shape = (n_rows, P_x)

# 1) compute all pairwise products with broadcasting:
#    this gives shape (n_rows, P_c, P_x)
K = C[:,:,None] * X[:,None,:]

# 2) reshape to (n_rows, P_c * P_x)
Z = K.reshape(len(df), -1)

# 3) build the column names in the same order
xc_names = [
    f"{c}_x_{m}"
    for c in numeric_cols + dummy_cols
    for m in macro_cols
]

# 4) wrap back into a DataFrame
df_xc = pd.DataFrame(Z, columns=xc_names, index=df.index)

In [ ]:
feature_cols = numeric_cols + xc_names + dummy_cols + nace_cols
nummeric_cols = numeric_cols + xc_names
cat_cols = dummy_cols + nace_cols
df_z = df_raw.merge(df_xc, left_index=True, right_index=True)
# drop macro_cols
df_z = df_z.drop(columns=macro_cols)
# sort columns by feature_cols
df_norm = df_z[['timestamp', 'ticker', 'target'] + feature_cols]


,timestamp,ticker,target,invest,cash,quick,maxret,retvol,turn,std_turn,...,NACE__65,NACE__66,NACE__68,NACE__70,NACE__72,NACE__77,NACE__80,NACE__81,NACE__82,NACE__93
0,2002-01-31,AAB.CO,0.120000,43.424000,0.317120,3.16768,0.040816,0.018440,0.008610,0.008442,...,0,0,0,0,0,0,0,0,0,1
1,2002-02-28,AAB.CO,0.125000,43.424000,0.317120,3.16768,0.058824,0.025166,0.016397,0.038772,...,0,0,0,0,0,0,0,0,0,1
2,2002-03-31,AAB.CO,-0.063492,43.424000,0.317120,3.16768,0.125000,0.040068,0.018084,0.015664,...,0,0,0,0,0,0,0,0,0,1
3,2002-04-30,AAB.CO,-0.169492,42.880000,0.317120,3.16768,0.035088,0.019353,0.019653,0.031524,...,0,0,0,0,0,0,0,0,0,1
4,2002-05-31,AAB.CO,-0.040816,42.880000,0.317120,3.16768,0.020833,0.021445,0.016952,0.012795,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45570,2024-08-31,ZELA.CO,-0.082910,62.975000,0.824779,6.22623,0.038198,0.027445,4.479898,7.897657,...,0,0,0,0,1,0,0,0,0,0
45571,2024-09-30,ZELA.CO,-0.027675,62.975000,0.824779,6.22623,0.049824,0.026599,3.886025,2.234252,...,0,0,0,0,1,0,0,0,0,0
45572,2024-10-31,ZELA.CO,-0.076534,63.138333,0.824779,6.22623,0.031209,0.017900,3.073600,0.622027,...,0,0,0,0,1,0,0,0,0,0
45573,2024-11-30,ZELA.CO,-0.019863,63.138333,0.824779,6.22623,0.061986,0.041247,2.320672,0.910035,...,0,0,0,0,1,0,0,0,0,0


## <a id='toc4_1_'></a>[Neural network data](#toc0_)
Including a validation set

In [4]:
# prepare containers
X_train, X_val, X_test = {}, {}, {}
y_train, y_val, y_test = {}, {}, {}
preprocessors = {}

for y, period in periods.items():
    period = pd.to_datetime(period)

    # split masks
    tr_mask = df_norm['timestamp'] < period
    va_mask = (df_norm['timestamp'] >= period) & \
              (df_norm['timestamp'] - pd.DateOffset(years=1) < period)
    te_mask = (df_norm['timestamp'] - pd.DateOffset(years=1) >= period) & \
              (df_norm['timestamp'] - pd.DateOffset(years=2) < period)

    # extract raw feature DataFrames
    X_tr_df = df_norm.loc[tr_mask, feature_cols].copy()
    X_va_df = df_norm.loc[va_mask, feature_cols].copy()
    X_te_df = df_norm.loc[te_mask, feature_cols].copy()
    y_tr    = y_values[tr_mask]
    y_va    = y_values[va_mask]
    y_te    = y_values[te_mask]

    # compute winsorization bounds on train
    lower = X_tr_df[numeric_cols].quantile(0.01)
    upper = X_tr_df[numeric_cols].quantile(0.99)

    # apply clipping to train, val, test
    X_tr_df[numeric_cols] = X_tr_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)
    X_va_df[numeric_cols] = X_va_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)
    X_te_df[numeric_cols] = X_te_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)


    # now fit scaler on numeric only
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), numeric_cols),
        ('cat', 'passthrough',  cat_cols)
    ])
    preprocessor.fit(X_tr_df)
    preprocessors[y] = preprocessor

    # transform all splits
    X_train[y] = preprocessor.transform(X_tr_df).astype('float32')
    X_val[y]   = preprocessor.transform(X_va_df).astype('float32')
    X_test[y]  = preprocessor.transform(X_te_df).astype('float32')

    # store targets as before
    y_train[y] = y_tr.reshape(-1, 1)
    y_val[y]   = y_va.reshape(-1, 1)
    y_test[y]  = y_te.reshape(-1, 1)


KeyError: "['sin'] not in index"

## <a id='toc4_2_'></a>[Linear model data](#toc0_)
Excluding the validation set

In [ ]:

Xlin_train, Xlin_test = {}, {}
ylin_train, ylin_test = {}, {}
preprocessors_lin = {}

cat_cols_lin = cat_cols.copy() + ['const']
feature_cols_lin = feature_cols + ['const']

for y, period in periods.items():
    period = pd.to_datetime(period)
    tr_mask = df_norm['timestamp']- pd.DateOffset(years=1) < period
    te_mask = (df_norm['timestamp'] - pd.DateOffset(years=1) >= period) & \
              (df_norm['timestamp'] - pd.DateOffset(years=2) < period)

    # extract feature DataFrames
    X_tr_df = df_norm.loc[tr_mask, feature_cols]
    X_te_df = df_norm.loc[te_mask, feature_cols]
    y_tr = y_values[tr_mask]
    y_te = y_values[te_mask]

    # add constant column for linear regression
    X_tr_df['const'] = 1
    X_te_df['const'] = 1

    # compute winsorization bounds on train
    lower = X_tr_df[numeric_cols].quantile(0.01)
    upper = X_tr_df[numeric_cols].quantile(0.99)

    # apply clipping to train, test
    X_tr_df[numeric_cols] = X_tr_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)
    X_te_df[numeric_cols] = X_te_df[numeric_cols].clip(lower=lower, upper=upper, axis=1)

    # fit scaler only on training set
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), numeric_cols),
        ('cat', 'passthrough', cat_cols_lin)
    ])
    preprocessor.fit(X_tr_df)
    preprocessors_lin[y] = preprocessor

    # ttransform splits
    Xlin_train[y] = preprocessor.transform(X_tr_df).astype('float32')
    Xlin_test[y]  = preprocessor.transform(X_te_df).astype('float32')

    # targets
    ylin_train[y] = y_tr
    ylin_test[y]  = y_te

# <a id='toc5_'></a>[Train Neural Network](#toc0_)

In [ ]:
# moving to metal or CUDA GPU if available
device = torch.device(("cuda" if torch.cuda.is_available() 
                       else "mps" if torch.backends.mps.is_available() 
                       else "cpu"))
print("Using device:", device)

# general hyperparameters
hidden_depth = 3 # only hidden, excluding in- and output layers
hidden_width = 32 # int for all being equal width; list for different widths
learning_rate = 1e-4
activation_fun = nn.ReLU # nn.ReLU nn.Tanh nn.Sigmoid nn.LeakyReLU

# general critereon and regularization parameters
criterion = nn.MSELoss()
lambda_l1 = 1e-3 # l1 regularization
lambda_l2 = 1e-3 # l2 regularization
dropout = 0.05

# general parmeters
patience = 25 # 10% of epochs
print_freq = 250
epochs = 250
batch_size = 4096 # 8192 # 16384 # 2^1X adjust to your memory


Using device: mps


In [ ]:
best_models = {}
history = {}
models = {}
mlp_pred = {}

for y, period in periods.items():
    np.random.seed(42)
    torch.manual_seed(42)
    print(f"Training MLP for year '{y}...")
    input_dim = X_train[y].shape[1]
    models[y] = MLPModel(input_dim,depth=hidden_depth,width=hidden_width, dropout=dropout, activation=activation_fun).to(device)
    optimizer = torch.optim.Adam(models[y].parameters(), lr=learning_rate)
    train = MLPdataset(X_train[y], y_train[y])
    val = MLPdataset(X_val[y], y_val[y])
    best_models[y], history[y] = train_mlp(train,          
                                                val,
                                                models[y],
                                                criterion,
                                                epochs,
                                                patience,
                                                print_freq,
                                                device,
                                                optimizer,
                                                lambda_l1=lambda_l1,
                                                lambda_l2=lambda_l2,
                                                batch_size=batch_size,
                                                shuffle_train=True,
                                                shuffle_val=False,
                                                save_path=f'models/mlp_y{y}_l1{lambda_l1}_l2{lambda_l2}_drop{dropout}_lr{learning_rate}_w{hidden_width}_d{hidden_depth}.pth'
                                                )
    mlp_pred[y] = predict_mlp(best_models[y],
                              X_test[y],
                              y_test[y],
                              batch_size=batch_size,
                              device=device)

Training MLP for year '21...
Early stopping at epoch 166
Best val loss: 1.98794E-02
Model saved to models/mlp_y21_l10.001_l20.001_drop0.05_lr0.0001_w32_d3.pth
Training MLP for year '22...
Early stopping at epoch 52
Best val loss: 1.21495E-02
Model saved to models/mlp_y22_l10.001_l20.001_drop0.05_lr0.0001_w32_d3.pth
Training MLP for year '23...
Early stopping at epoch 99
Best val loss: 1.00447E-01
Model saved to models/mlp_y23_l10.001_l20.001_drop0.05_lr0.0001_w32_d3.pth
Training MLP for year '24...
Early stopping at epoch 72
Best val loss: 1.30650E-02
Model saved to models/mlp_y24_l10.001_l20.001_drop0.05_lr0.0001_w32_d3.pth


## <a id='toc5_1_'></a>[Graph the model](#toc0_)

In [ ]:
# for y, period in periods.items():
#     # plot training history
#     plt.figure(figsize=(10, 6))
#     plt.plot(history[y]['train_loss'], label='Train Loss')
#     plt.plot(history[y]['val_loss'],   label='Validation Loss')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.title(f'Training History for 2002-20{y}')
#     plt.legend()
#     plt.tight_layout()
#     plt.show()
#     plt.close()

# <a id='toc6_'></a>[Train linear models](#toc0_)

## <a id='toc6_1_'></a>[OLS](#toc0_)

In [ ]:
# linear model
# estimate the parameters
ols_est = {}
# ols_pred_train = {}
ols_pred = {}
ols_coeffs = {}


for y in periods.keys():
    print(f"Estimating OLS for {y}...")
    x_tr = Xlin_train[y]
    y_tr = ylin_train[y]
    x_te = Xlin_test[y]
    y_te = ylin_test[y]


    # estimate the parameters
    ols_est[y] = estimate(y_tr, x_tr)
    # ols_pred_train[y] = ols_est[y]['b_hat'] @ x_tr.T
    ols_pred[y] = ols_est[y]['b_hat'] @ x_te.T
    ols_coeffs[y] = ols_est[y]['b_hat']

Estimating OLS for 21...
Estimating OLS for 22...
Estimating OLS for 23...
Estimating OLS for 24...


/Users/johan/Documents/04 Uni/10 Thesis/git/MastersThesis/libs/functions.py:192: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(np.diag(cov)).reshape(-1, 1)


In [ ]:
ols_coeffs_df = pd.DataFrame(ols_coeffs, index=feature_cols_lin)
ols_coeffs_df = ols_coeffs_df.reindex(ols_coeffs_df.abs().sum(axis=1).sort_values(ascending=False).index, axis=0)
ols_coeffs_df = ols_coeffs_df
display(ols_coeffs_df)

,21,22,23,24
roaq,0.274466,-0.175469,-0.145786,-0.050596
roic,-0.275177,0.173897,0.143558,0.049338
NACE__27,0.063545,0.061786,0.116791,0.099426
NACE__77,-0.044534,-0.062176,-0.051485,-0.043641
NACE__81,-0.054414,-0.049405,-0.043303,-0.035333
...,...,...,...,...
lgr,-0.000362,-0.000218,-0.000401,-0.000389
pchgm_pchsale,0.000330,0.000272,0.000216,0.000259
pricedelay,-0.000012,-0.000141,-0.000736,-0.000160
salecash,0.000189,0.000244,0.000037,0.000385


## <a id='toc6_2_'></a>[LASSO](#toc0_)

In [ ]:
# linear model
# create a grid using numpy.geomspace
penalty_grid = np.geomspace(1e-7, 100, num = 1000)
lasso_est = {}
# lasso_pred_train = {}
lasso_pred = {}
lasso_coeffs = {}


with warnings.catch_warnings():
    warnings.simplefilter("ignore", ConvergenceWarning)
    for y in periods.keys():
        print(f"Estimating LASSO for {y}...")
        x_tr = Xlin_train[y]
        y_tr = ylin_train[y]
        x_te = Xlin_test[y]
        y_te = ylin_test[y]

        # estimate the model using LassoCV
        fit_CV = LassoCV(cv=5, alphas=penalty_grid, max_iter=1000, eps=1e-3, n_jobs=-1).fit(x_tr,y_tr)
        # lasso_pred_train[y] = fit_CV.predict(x_tr)
        lasso_pred[y] = fit_CV.predict(x_te)

        # store the coefficients
        coeff = fit_CV.coef_
        lasso_coeffs[y] = coeff


Estimating LASSO for 21...
Estimating LASSO for 22...
Estimating LASSO for 23...
Estimating LASSO for 24...


In [ ]:
lasso_coeffs_df = pd.DataFrame(lasso_coeffs, index=feature_cols_lin)
# sort by absolute value
lasso_coeffs_df = lasso_coeffs_df.reindex(lasso_coeffs_df.abs().sum(axis=1).sort_values(ascending=False).index, axis=0)

# drop columns with all zeros
lasso_coeffs_df = lasso_coeffs_df.T
lasso_coeffs_df = lasso_coeffs_df.loc[:, (lasso_coeffs_df != 0).any(axis=0)]
display(lasso_coeffs_df)

,mom12m,discount,cfp_ia,indmom,mve_ia,ep,age,idiovol,egr,dy,...,mom1m,pchquick,acc,mvel1,dp,rd_sale,lev,pchsale_pchinvt,quick,gma
21,0.008319,-0.009985,0.003633,0.002978,-0.003289,0.001755,-0.003924,-0.001181,-0.001032,0.000833,...,0.000911,0.000746,-0.000737,-0.000000,0.000451,-0.000367,-0.000322,-0.000356,-0.000169,0.000059
22,0.008465,-0.009390,0.003650,0.003125,-0.003308,0.002846,-0.002765,-0.001555,-0.001382,0.000904,...,0.001068,0.000836,-0.000652,-0.000000,0.000425,-0.000456,-0.000491,-0.000379,-0.000000,0.000000
23,0.008032,-0.006365,0.003731,0.003691,-0.002417,0.002520,-0.000000,-0.001658,-0.001484,0.001678,...,0.000143,0.000442,-0.000209,-0.000905,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
24,0.007615,-0.005886,0.003620,0.003560,-0.002591,0.001923,-0.000000,-0.001656,-0.001438,0.001563,...,0.000361,0.000383,-0.000184,-0.000547,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000


# Summarize the results

In [ ]:
pred_dfs = []

for y, period in periods.items():
    # rebuild masks
    te_mask = ((df_norm['timestamp'] - pd.DateOffset(years=1) >= period) &
               (df_norm['timestamp'] - pd.DateOffset(years=2) <  period))
    X_te_df = df_norm.loc[te_mask, feature_cols]
    idx = X_te_df.index


    pred_dfs.append(pd.DataFrame({
        'period':    y,
        'timestamp': df_norm.loc[idx, 'timestamp'],
        'ticker':    df_norm.loc[idx, 'ticker'],
        'y_true':    df_norm.loc[idx, 'target'].values.astype('float32'),
        'OLS':    ols_pred[y].flatten(),
        'LASSO':  lasso_pred[y],
        'MLP':    mlp_pred[y],
    }, index=idx))

all_preds = pd.concat(pred_dfs).sort_index()

In [ ]:
methods = ['OLS', 
           'LASSO', 
           'MLP',
           ]

results = {}

for method in methods:
    rmse = rmse_fun(all_preds[method], all_preds['y_true'])
    mae = mae_fun(all_preds[method], all_preds['y_true'])
    results[method] = {
        'RMSE': rmse,
        'MAE': mae
    }

metrics = {}
for metric in ['RMSE','MAE']:
    key = f'{metric}'
    vals = [
        results['OLS'][metric],
        results['LASSO'][metric],
        results['MLP'][metric],
        ]
    metrics[key] = vals

tab = latex_table(methods,metrics)

print(tab)

\begin{tabular}{lccc}
\hline\hline \\ [-1.8ex]
 & OLS & LASSO & MLP \\ 
 \hline 
RMSE & 0.19615 & 0.19447 & 0.19460 \\ 
MAE & 0.08069 & 0.07740 & 0.07806 \\ 
\hline\hline
\end{tabular}


In [ ]:
metrics_full = {}
results_full = {}
for y, period in periods.items():
    for method in methods:
        y_true = all_preds.loc[all_preds['period'] == y, 'y_true']
        y_pred = all_preds.loc[all_preds['period'] == y, method]
        key = f'{method}{y}'
        results_full[key] = {
            'RMSE': rmse_fun(y_pred, y_true),
            'MAE': mae_fun(y_pred, y_true)
        }
    
    for metric in ['RMSE','MAE']:
        key = f'*{metric}*20{y}'
        vals = [
            results_full[f'OLS{y}'][metric],
            results_full[f'LASSO{y}'][metric],
            results_full[f'MLP{y}'][metric],
        ]
        metrics_full[key] = vals

tab_full = latex_table_grouped(methods,metrics_full)

print(tab_full)


\begin{tabular}{clccc}
\hline\hline \\ [-1.8ex]
 &  & OLS & LASSO & MLP \\ 
 \hline 
\multirow[c]{4}{*}{\rotatebox{90}{RMSE}}& 2021 & 0.10989 & 0.10936 & 0.11064 \\ 
 & 2022 & 0.31726 & 0.31720 & 0.31720 \\ 
 & 2023 & 0.11853 & 0.11355 & 0.11442 \\ 
 & 2024 & 0.16387 & 0.15966 & 0.15870 \\ 
\hline\multirow[c]{4}{*}{\rotatebox{90}{MAE}}& 2021 & 0.07319 & 0.07263 & 0.07330 \\ 
 & 2022 & 0.09793 & 0.09664 & 0.09721 \\ 
 & 2023 & 0.07441 & 0.06845 & 0.06924 \\ 
 & 2024 & 0.07717 & 0.07162 & 0.07220 \\ 
\hline\hline
\end{tabular}
